## Scrapy的Scrapyd详解
　　参考：[官方文档](https://scrapyd.readthedocs.io/en/stable/api.html)<br>
　　　　　[配置文档](https://scrapyd.readthedocs.io/en/latest/config.html)<br>
　　　　　[Scrapyd使用详解](https://www.jianshu.com/p/9a4ce1e2fa7b)<br>
　　　　　[]()<br>

### 一、Scrapyd介绍
　　scrapyd 是由scrapy 官方提供的爬虫管理工具（需要单独安装），使用它我们可以非常方便地上传、控制爬虫并且查看运行日志。<br>
　　使用scrapyd 和我们直接运行scrapy crawl myspider有什么区别呢？<br>
　　scrapyd 同样是通过上面的命令运行爬虫的，不同的是它提供一个JSON web service 监听的请求，我们可以从任何一台可以连接到服务器的电脑发送请求安排爬虫运行，或者停止正在运行的爬虫。甚至，我们可以使用它提供的API上传新爬虫而不必登录到服务器上进行操作。<br>
　　Scrapyd通常作为守护进程运行，它侦听运行爬虫的请求，并为每个请求生成一个进程，该进程基本上执行:scrapy crawl [myspider]。<br>
　　Scrapyd还并行运行多个进程，将它们分配到max_proc和max_proc_per_cpu选项提供的固定数量的插槽中，启动尽可能多的进程来处理负载。<br>
　　Scrapyd的web界面比较简单，主要用于监控，所有的调度工作全部依靠接口实现.
### 二、Scrapyd安装配置
#### 1、安装
>pip install scrapyd<br>
#### 2、运行Scrapyd服务
　　直接运行命令scrapyd即可：
>scrapyd<br>
#### 3、修改默认配置信息
　　官方详细配置文档说明：https://scrapyd.readthedocs.io/en/latest/config.html<br>
　　可以在项目下新建一个scrapyd.conf或者在scrapy.cfg中增加[scrapyd]：<br>
>[scrapyd]              
bind_address = 127.0.0.1　　　# 网页和Json服务监听的IP地址，默认为127.0.0.1<br>
http_port = 6800　　　　　　　# 监听的端口，默认为6800<br>
debug  = off　　　　　　　　　# 是否打开debug模式，默认为off<br>
max_proc_per_cpu = 4　　　　　# 每个CPU可启用的Scrapy 进程数，默认为4<br>
max_proc = 0　　　　　　　　　# 可启用的最多进程数，默认为0.如果未设置或者设为0，则使用的最多进程数=CPU数量*max_proc_per_cpu<br>
eggs_dir = eggs　　　　　　　# 项目eggs生成目录，默认为项目目录下eggs<br>
logs_dir = logs<br>
items_dir = ''　　　　　　　　# 项目日志生成目录，默认为项目目录下logs，如果不想要生成日志，可以直接设置成空<br>
dbs_dir = dbs　　　　　　　　# 项目dbs生成目录，默认为项目目录下dbs<br>
items_dir = ''　　　　　　　# 爬取的items存储的文件夹（版本0.15.以上），默认为空，不存储。<br>
jobs_to_keep = 5　　　　　　　# 每个爬虫保持的完成任务数，默认为5.（版本0.15.以上，以前版本中为logs_to_keep）<br>
finished_to_keep = 100　　　　# 保持的完成任务进程数。默认为100.（版本0.14.以上）<br>
poll_interval = 5.0　　　　　# 轮训请求队列的时间间隔。默认为5s，可以为浮点数<br>
runner = scrapyd.runner   　# 启动子进程的模块。可以使用自定义 <br>
application = scrapyd.app.application　　　　# 返回可用于twisted的application，可继承于Scrapyd添加和移除自己的组件和服务。https://twistedmatrix.com/documents/current/core/howto/application.html查看更多<br>
launcher = scrapyd.launcher.Launcher    # twisted的web资源，表示到scrapyd的接口。Scrapyd包含一个带有网站的界面，可以提供对应用程序的web资源的简单监视和访问。此设置必须提供twisted web资源的根类。<br>
webroot  = scrapyd.website.Root<br>


　　默认情况下scrapyd 监听 0.0.0.0:6800 端口，运行 scrapyd 后在浏览器http://localhost:6800/ 即可查看到当前可以运行的项目。<br>
　　web接口：<br>
>http://localhost:6800/

### 三、Scrapyd使用
#### 1、部署scrapy 项目
　　**Scrapyd部署主要分为两步：**
>1、将项目打包成egg<br>
2、将egg通过Scrapyd的addversion.json接口上传到目标服务器<br>

　　官方的部署上传接口文档：https://scrapyd.readthedocs.io/en/latest/api.html#addversion-json

　　推荐**使用Scrapyd-client中的 scrapyd-deploy 打包上传**一步到位，关于Scrapyd-client介绍：
>Scrapyd-client is a client for Scrapyd. It provides the general scrapyd-client and the scrapyd-deploy utility which allows you to deploy your project to a Scrapyd server.

　**Scrapy-client安装:**
>pip install scrapyd-client

　　直接在项目根目录,修改工程目录下的 scrapy.cfg 文件：
>[deploy:scrapyd2]　　　　　# 默认情况下并没有scrapyd2，它只是一个名字，可以在配置文件中写多个名字不同的deploy<br>
url = http://scrapyd.mydomain.com/api/scrapyd/　　　  # 要部署项目的服务器的地址<br>
username = john 　　　　　　　　　　 # 访问服务器所需的用户名和密码（如果不需要密码可以不写）<br>
password = secret<br>

　　其中的username 和 password 用于在部署时验证服务器的HTTP basic authentication，须要注意的是这里的用户密码并不表示访问该项目须要验证，而是登录服务器用的。<br>
　　Ubuntu/Windows:
>[deploy:tutorial_deploy]<br>
url = http://192.168.17.129:6800/<br>
project = tutorial<br>
username = enlong<br>
password = test<br>

　　**部署项目到服务器**,直接在项目根目录(即scrapy.cfg所在的目录下运行:scrapyd-deploy)：
　　Windows:
>python c:\Python27\Scripts\scrapyd-deploy

　　Ubuntu:
>scrapyd-deploy tutorial_deploy -p tutorial

　　部署操作会打包你的当前项目，如果当前项目下有setup.py文件，就会使用它，没有的会就会自动创建一个(如果后期项目需要打包的话，可以根据自己的需要修改里面的信息，也可以暂时不管它)。从返回的结果里面，可以看到部署的状态，项目名称，版本号和爬虫个数，以及当前的主机名称。<br>
　　注意：部署前，需要先在项目目录下运行scrapyd，启动Scrapyd服务。

#### 2、查看项目spider
　　通过scrapyd-deploy -l 查看当前目录下的可以使用的部署方式（target）<br>
　　Windows/Ubuntu:
>scrapy list<br>
scrapyd-deploy -l

　　或再次打开http://localhost:6800/, 也可以看到Available projects: default, tutorial。列出服务器上所有的项目，检查tutorial_deploy是否已经部署上去了:
>scrapyd-deploy -L tutorial_deploy<br>
default<br>
tutorial

#### 3、开启爬虫 schedule
>curl http://localhost:6800/schedule.json -d project=tutorial -d spider=tencent

　　Windows/Ubuntu 注意：执行时 cd 到项目根目录执行
>curl http://localhost:6800/schedule.json -d project=tutorial -d spider=tencent<br>
{"status": "ok", "jobid": "94bd8ce041fd11e6af1a000c2969bafd", "node_name": "ubuntu"}　　　　# 返回状态值

#### 4、停止 cancel
>curl http://localhost:6800/cancel.json -d project=tutorial -d job=94bd8ce041fd11e6af1a000c2969bafd

#### 5、列出爬虫
>curl http://localhost:6800/listspiders.json?project=tutorial

#### 6、删除项目
>curl http://localhost:6800/delproject.json -d project=tutorial

#### 7、更新
　　对于Scrapyd默认项目(即是启动Scrapyd命令后看到的default项目)：<br>
　　只有在scrapy项目里启动scrapyd命令时才有默认项目，默认项目就是当前的scrapy项.如果在非scrapy项目下执行scrapyd, 是看不到default的<br>
　　注意：执行时 cd 到项目根目录执行<br>
　　第一种情况cfg:
>[deploy]<br>
url = http://192.168.17.129:6800/<br>
project = tutorial<br>
username = enlong<br>
password = test

　　运行结果：
>python@ubuntu:~/project/tutorial$ scrapyd-deploy <br>
Packing version 1471069533<br>
Deploying to project "tutorial" in http://192.168.17.129:6800/addversion.json<br>
Server response (200):<br>
{"status": "ok", "project": "tutorial", "version": "1471069533", "spiders": 1, "node_name": "ubuntu"}<br>

　　第二种情况cfg:
>[deploy:tutorial_deploy]<br>
url = http://192.168.17.129:6800/<br>
project = tutorial<br>
username = enlong<br>
password = test

　　运行结果：
>python@ubuntu:~/project/tutorial$ scrapyd-deploy tutorial_deploy<br>
Packing version 1471069591<br>
Deploying to project "tutorial" in http://192.168.17.129:6800/addversion.json<br>
Server response (200):<br>
{"status": "ok", "jobid": "94bd8ce041fd11e6af1a000c2969bafd", "node_name": "ubuntu"}<br>

